#### Task 1: Create a custom password validator

Django defines a password policy using the AUTH_PASSWORD_VALIDATORS setting, you can create a custom password validator and add it as an entry to this setting. The purpose of this validator is to check if a password consist of 3 words and if the words are contained in the password dictionary.

> If you cannot find /usr/share/dict/words in your filesystem, run ``sudo apt-get install --reinstall wamerican``

* Start by creating a new file named **validators.py** in the profile_info application directory
* Create a new class named **PassphraseValidator** using the code below:
    > ```python
        from django.core.exceptions import ValidationError
        from django.utils.translation import gettext_lazy as _

        class PassphraseValidator:
            def __init__(self, dictionary_file='/usr/share/dict/words'):
                self.min_words = 3
                with open(dictionary_file) as f: 
                    self.words = set(word.strip() for word in f) 

            def get_help_text(self):
                return _('Your password must contain %s words' % self.min_words) 

            def validate(self, password, user=None):
                tokens = password.split(' ')
                if len(tokens) < self.min_words: 
                    too_short = _('This password needs %s words' % self.min_words) 
                    raise ValidationError(too_short, code='too_short') 
                if not all(token in self.words for token in tokens): 
                    not_passphrase = _('This password is not a passphrase') 
                    raise ValidationError(not_passphrase, code='not_passphrase')
    ```

* Open **settings.py** and add the following entry to the AUTH_PASSWORD_VALIDATORS list:
    > ```python
        {
            'NAME': 'profile_info.validators.PassphraseValidator'
        }

    ```

* Restart the Django server and navigate to https://localhost:8000/accounts/password_change
* Notice the additional password requirement. **Your password must contain 3 words**

#### Task 2: Creating a custom password hasher

Django uses Password-Based Key Derivation Function 2 (PBKDF2) to hash passwords. PBKDF2 has a configurable iteration count and hash algorithm. In this task, you will create custom password hasher that doubles the iteration count for PBKDF2.

* In the profile_info application directory, add a new file named **hashers.py**
* Open hashers.py and add the code below:
    >```python
        from django.contrib.auth.hashers import PBKDF2PasswordHasher
        
        class TwoFoldPBKDF2PasswordHasher(PBKDF2PasswordHasher):
            iterations = PBKDF2PasswordHasher.iterations * 2 
    ```